In [2]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

import re

def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []

wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16).map(parse_article)

In [ ]:
from operator import add
bigrams = wiki.flatMap(lambda x: [((x[i],x[i+1]),1) for i in range(0,len(x)-1)]).reduceByKey(add)
results = bigrams.filter(lambda x: "narodnaya" in x[0][0].lower()).collect()

In [ ]:
d = {}
for r in results:
    key = r[0][0].lower() + "_" + r[0][1].lower() 
    d[key] = r[1]
for key, value in sorted(d.items()):
    print(key + "\t" + str(value))